In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from comtrade import Comtrade
import seaborn as sns

##### Functions to be used

In [ ]:
def data_frames_creation(paths):
    temp=[]
    for i in range (0, len(paths)):
        rec = Comtrade()
        rec.load(paths[i][0],paths[i][1])
        samples_list=[]
        for j in range (0, len(rec.analog)):
            samples_list.append(rec.analog[j])
        abc=pd.DataFrame(samples_list)
        abc=abc.transpose()
        abc.columns=rec.analog_channel_ids
        time=pd.Series(data=rec.time,name='Time')
        abc=pd.concat([time,abc],axis=1)
        temp.append(abc)
        samples_list=[]
    return temp

In [ ]:
import os
def getPath(file_id, file_type):
    response = ''
    directory = "C:\\Users\\Mahnoor Saeed\\Dissertation_Notebook\\cable_failure_dataset\\DAT_Files"
    if file_type == 2:
        directory = "C:\\Users\\Mahnoor Saeed\\Dissertation_Notebook\\cable_failure_dataset\\CFG_FILES"
    for path in os.listdir(directory):
        full_path = os.path.join(directory, path)
        if os.path.isfile(full_path):
            if str(file_id) in full_path:
                response = full_path
    return response

In [ ]:
def pred_fal_good_Visualization(dataframe, wrong_phases,columns):
    for f in range (0,len(dataframe)):
        if wrong_phases[f]=='A':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[0])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:I A'],color='#E9967A')
        if wrong_phases[f]=='B':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[1])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:I B'],color='#108A0D')
        if wrong_phases[f]=='C':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[2])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:I C'],color='#0282B9')
        if wrong_phases[f]=='N':
            plt.figure(figsize=(15,3))
            plt.title('Failure Sample '+str(f)+' Amplitude Vs Time of '+columns[3])
            plt.plot(dataframe[f]['Time'],dataframe[f]['MPI3p1:IN'],color='#C794F2')

###### `********************************************************************************************************************`

In [ ]:
ten_PCA=pd.read_csv('Ten_com_clustering.csv')

In [ ]:
ten_PCA.head()

#### GMM-Diagonal

In [ ]:
ten_PCA

In [ ]:
np.random.seed(40)
from sklearn.mixture import GaussianMixture
gmm_diag = GaussianMixture(n_components=2,covariance_type='diag').fit(ten_PCA)

In [ ]:
gmm_diag.get_params()

In [ ]:
np.random.seed(40)
gmm_dia_labels=gmm_diag.predict(ten_PCA)

In [ ]:
gmm_dia_df = pd.concat([ten_PCA, pd.DataFrame({'pca_clusters':gmm_dia_labels})], axis=1)

In [ ]:
from sklearn.metrics import silhouette_score
print('Silhoutte score of GMM clustering is ' + str(silhouette_score(ten_PCA, gmm_dia_labels)))

In [ ]:
gmm_dia_df.head()

In [ ]:
gmm_dia_df.to_csv('supervised_learning_dataset.csv',index = False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
gmm_dia_df['pca_clusters'].value_counts()

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(gmm_dia_df.iloc[:,0],gmm_dia_df.iloc[:,1], hue=gmm_dia_df['pca_clusters'], palette='mako', s=70, alpha=0.8).set_title('Distribution of cluster Based on Gaussian Mixture Model (Diagonal)',fontsize=13)
plt.legend()
plt.xlabel('1st Principal Component')
plt.ylabel('2nd Principal Component')
plt.savefig('Gmm_result')
plt.show()

###### `********************************************************************************************************************`

## Predicted Failure Verification by GMM Diagonal 

In [ ]:
pred_fal_gmm_dia=gmm_dia_df[gmm_dia_df['pca_clusters']==1]

In [ ]:
pred_fal_gmm_dia

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(pred_fal_gmm_dia.iloc[:,0],pred_fal_gmm_dia.iloc[:,1],color='#08BF4B' , s=70, alpha=0.8).set_title('Only Predicted Failure cluster by GMM',fontsize=13)
plt.legend(labels=['Predicted Failure Cluster Only'])
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
predicted_failure_gmm_index=pred_fal_gmm_dia.index

In [ ]:
engin_df=pd.read_csv('eng_features.csv')
engin_df.head()

In [ ]:
Faulty_pred_gmm_ids=[]
for i in predicted_failure_gmm_index:
    Faulty_pred_gmm_ids.append(engin_df.loc[i])
Faulty_pred_gmm_ids=pd.DataFrame(Faulty_pred_gmm_ids)

In [ ]:
Faulty_pred_gmm_ids

In [ ]:
len(Faulty_pred_gmm_ids)

In [ ]:
gmm_dia_failure_paths=[]
for index, row in Faulty_pred_gmm_ids.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    gmm_dia_failure_paths.append([CFG_Path, dAT_path])

In [ ]:
gmm_dia_faulty_phases=[]
for index, row in Faulty_pred_gmm_ids.iterrows():
    gmm_dia_faulty_phases.append(row['Phase'])

In [ ]:
print(len(gmm_dia_failure_paths))
print(len(gmm_dia_faulty_phases))

In [ ]:
gmm_dia_fal_data=data_frames_creation(gmm_dia_failure_paths)

In [ ]:
len(gmm_dia_fal_data)

In [ ]:
col_names=[ 'MPI3p1:I A', 'MPI3p1:I B', 'MPI3p1:I C', 'MPI3p1:IN']

In [ ]:
pred_fal_good_Visualization(gmm_dia_fal_data,gmm_dia_faulty_phases,col_names)

In [ ]:
false_failure_diag=[2,32,34,52,60,78,134,153,275,293,320]

In [ ]:
len(false_failure_diag)

In [ ]:
len(pred_fal_gmm_dia)

In [ ]:
predicted_failure_gmm_dia_reset=pred_fal_gmm_dia.reset_index(drop=True)

In [ ]:
predicted_failure_gmm_dia_reset

In [ ]:
fal_drop_gmm_dia=pred_fal_gmm_dia.iloc[false_failure_diag]

In [ ]:
len(fal_drop_gmm_dia)

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(pred_fal_gmm_dia.iloc[:,0],pred_fal_gmm_dia.iloc[:,1], color='#677666', s=70, alpha=0.4).set_title('Predicted Failure cluster only based on GMM (Co-Variance type: Diagonal)',fontsize=13)
sns.scatterplot(fal_drop_gmm_dia.iloc[:,0],fal_drop_gmm_dia.iloc[:,1],  s=70, alpha=0.8, color='#F8652A')
#plt.legend()
plt.legend(labels=["True Failure","False Failure"])
#plt.savefig('failure_only')
plt.show()

## Predicted Good Verification

In [ ]:
predicted_good_gmm_dia=gmm_dia_df[gmm_dia_df['pca_clusters']==0]

In [ ]:
predicted_good_gmm_dia.head()

In [ ]:
Gmm_dia_predicted_good_index=predicted_good_gmm_dia.index
Gmm_dia_predicted_good_index

In [ ]:
len(Gmm_dia_predicted_good_index)

In [ ]:
Good_pred_ids_gmm_dia=[]
for i in Gmm_dia_predicted_good_index:
    Good_pred_ids_gmm_dia.append(engin_df.loc[i])
Good_pred_ids_gmm_dia=pd.DataFrame(Good_pred_ids_gmm_dia)

In [ ]:
Good_pred_ids_gmm_dia.head()

In [ ]:
gmm_dia_paths_g=[]
for index, row in Good_pred_ids_gmm_dia.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    gmm_dia_paths_g.append([CFG_Path, dAT_path])

In [ ]:
gmm_dia_g_phases=[]
for index, row in Good_pred_ids_gmm_dia.iterrows():
    gmm_dia_g_phases.append(row['Phase'])

In [ ]:
len(gmm_dia_g_phases)

In [ ]:
pred_good_gmm_dia=data_frames_creation(gmm_dia_paths_g)

In [ ]:
len(pred_good_gmm_dia)

In [ ]:
pred_fal_good_Visualization(pred_good_gmm_dia,gmm_dia_g_phases,col_names)

In [ ]:
predicted_good_gmm_dia.head()

In [ ]:
gmm_dia_false_g=[93,99]

In [ ]:
predicted_good_gmm_dia_index_resset=predicted_good_gmm_dia.reset_index(drop=True)

In [ ]:
good_pred_false_df=predicted_good_gmm_dia_index_resset.iloc[gmm_dia_false_g]

In [ ]:
good_pred_false_df

In [ ]:
len(predicted_good_gmm_dia)

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_good_gmm_dia.iloc[:,0],predicted_good_gmm_dia.iloc[:,1], color='#677666',  s=70, alpha=0.8).set_title(' Only Predicted Good cluster based GMM (Diagonal)',fontsize=13)
plt.legend(labels=['Predicted Good Cluster Only'],loc='upper left')
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(predicted_good_gmm_dia.iloc[:,0],predicted_good_gmm_dia.iloc[:,1], color='#677666', s=70, alpha=0.8).set_title(' True Positive Vs False Positive Baes on GMM (Diagonal)',fontsize=13)
sns.scatterplot(good_pred_false_df.iloc[:,0],good_pred_false_df.iloc[:,1], color='#F8652A', s=70, alpha=0.8)
#plt.legend()
plt.legend(labels=["True Positive","False Positive"])
#plt.savefig('good_pred')
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize = (20,7))
sns.scatterplot(pred_fal_gmm_dia.iloc[:,0],pred_fal_gmm_dia.iloc[:,1], color='#677666', s=70, alpha=0.6).set_title('Predicted Failure cluster only based on GMM',fontsize=13)
sns.scatterplot(fal_drop_gmm_dia.iloc[:,0],fal_drop_gmm_dia.iloc[:,1],  s=70, alpha=0.8, color='#F8652A')

sns.scatterplot(predicted_good_gmm_dia.iloc[:,0],predicted_good_gmm_dia.iloc[:,1], color='#7926FA', s=70, alpha=0.8).set_title(' Gaussian Mixture Model (Diagonal)',fontsize=13)
sns.scatterplot(good_pred_false_df.iloc[:,0],good_pred_false_df.iloc[:,1], color='#26FAA0', s=70, alpha=0.8)
plt.legend(labels=["True Failure","False Failure","True Positive","False Positive"])

#plt.xlim(-5, )
#plt.ylim(-40, 40)
#plt.savefig('failure_only')

plt.show()

#### GMM Diagonal Evaluation

In [ ]:
pred_failure_diag=404
false_failure_diagonal=11
pred_good_diagonal=968
incor_good_preds_diagonal=2

In [ ]:
True_negative_diag=404-11+2

In [ ]:
True_negative_diag

In [ ]:
True_positive_diag=968-2+11
True_positive_diag

In [ ]:
Precision_Gmm_Diagonal_good=(True_positive_diag)/(True_positive_diag+incor_good_preds_diagonal)

In [ ]:
Precision_Gmm_Diagonal_good

In [ ]:
recall_Gmm_Diagonal_good=(True_positive_diag)/(True_positive_diag+false_failure_diagonal)

In [ ]:
recall_Gmm_Diagonal_good

In [ ]:
F1_Gmm_Diagonal_good=(2*Precision_Gmm_Diagonal_good*recall_Gmm_Diagonal_good)/(Precision_Gmm_Diagonal_good+recall_Gmm_Diagonal_good)

In [ ]:
F1_Gmm_Diagonal_good

In [ ]:
Precision_Gmm_Diagonal_failure=(True_negative_diag)/(True_negative_diag+incor_good_preds_diagonal)

In [ ]:
recall_gmm_failure=(True_negative_diag)/(True_negative_diag+false_failure_diagonal)
recall_gmm_failure

In [ ]:
F1_Gmm_Diagonal_failure=(2*Precision_Gmm_Diagonal_failure*recall_gmm_failure)/(Precision_Gmm_Diagonal_failure+recall_gmm_failure)

In [ ]:
F1_Gmm_Diagonal_failure

In [ ]:
# Accuracy Formula=(TP+TN)/(TP+FN+TN+FP)

In [ ]:
Accuracy_Gmm_Diagonal=(True_positive_diag+True_negative_diag)/(True_positive_diag+false_failure_diagonal+True_negative_diag+incor_good_preds_diagonal)

In [ ]:
Accuracy_Gmm_Diagonal

In [ ]:
#Specificity=(TN/(TN+FP))

In [ ]:
Specificity_Gmm_Diagonal=(True_negative_diag)/(True_negative_diag+incor_good_preds_diagonal)

In [ ]:
Specificity_Gmm_Diagonal

In [ ]:
Evaluation_df_gmm=pd.DataFrame({
    'Precision':[Precision_Gmm_Diagonal_good,Precision_Gmm_Diagonal_failure],
    'Recall': [recall_Gmm_Diagonal_good,recall_gmm_failure],
    'F1-Score':[F1_Gmm_Diagonal_good,F1_Gmm_Diagonal_failure],
},
index=['Good Condition (GMM)','Failure (GMM)'])

In [ ]:
Evaluation_df_gmm

#### EVALUATION WITH RANDOM STATES

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score

In [ ]:
random_states=[32,199,280,500,1000,2000]

In [ ]:
sil_score=[]
cal_score=[]
dav_bold=[]

In [ ]:
for i in random_states:
    gmm_diag_eval = GaussianMixture(n_components=2,covariance_type='diag',random_state=i).fit(ten_PCA)
    gmm_dia_labels_eval=gmm_diag_eval.predict(ten_PCA)
    sil_score.append(silhouette_score(ten_PCA,gmm_dia_labels_eval)) 
    cal_score.append(calinski_harabasz_score(ten_PCA,gmm_dia_labels_eval))
    dav_bold.append(davies_bouldin_score(ten_PCA,gmm_dia_labels_eval))

In [ ]:
sil_score

In [ ]:
cal_score

In [ ]:
dav_bold

In [ ]:
index_df=pd.DataFrame({'sil':sil_score,
                      'calinski':cal_score,
                     'DB':dav_bold})

In [ ]:
index_df

In [ ]:
index_df_drop_calin=index_df.drop('calinski',axis=1)

In [ ]:
index_df_drop_calin

###### Experiment (Clustering the Failure)

In [ ]:
Faulty_pred_gmm_ids

In [ ]:
Faulty_pred_gmm_ids_rest=Faulty_pred_gmm_ids.reset_index(drop=True)

In [ ]:
Faulty_pred_gmm_ids_rest

In [ ]:
false_failure_diag

In [ ]:
only_pred_fal_gmm=Faulty_pred_gmm_ids_rest.drop(false_failure_diag,axis=0)

In [ ]:
only_pred_fal_gmm

In [ ]:
only_gmm_dia_failure_paths=[]
for index, row in only_pred_fal_gmm.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    only_gmm_dia_failure_paths.append([CFG_Path, dAT_path])

In [ ]:
only_gmm_dia_faulty_phases=[]
for index, row in only_pred_fal_gmm.iterrows():
    only_gmm_dia_faulty_phases.append(row['Phase'])

In [ ]:
print(len(only_gmm_dia_failure_paths))
print(len(only_gmm_dia_faulty_phases))

In [ ]:
pred_fal_exp_data=data_frames_creation(only_gmm_dia_failure_paths)

In [ ]:
pred_fal_exp_data

In [ ]:
exp_dataset_1=predicted_failure_gmm_dia_reset.drop(false_failure_diag,axis=0)

In [ ]:
exp_dataset_1=exp_dataset_1.drop('pca_clusters',axis=1)

In [ ]:
exp_dataset_1

In [ ]:
np.random.seed(43)
from sklearn.cluster import KMeans

kmeans_models_s = [KMeans(n_clusters=k).fit(exp_dataset_1) for k in range (1, 10)]
innertia_s = [model.inertia_ for model in kmeans_models_s]

plt.plot(range(1, 10), innertia_s,'-o',color='#783654')
plt.title('Elbow method (Spectral Domian Features)')
plt.xlabel('Number of Clusters')
plt.ylabel('Model Inertia')
plt.show()
#plt.savefig('Elbow_1.png');

In [ ]:
np.random.seed(40)
from sklearn.metrics import silhouette_score

silhoutte_scores_s = [silhouette_score(exp_dataset_1, model.labels_) for model in kmeans_models_s[1:10]]
plt.plot(range(2,10), silhoutte_scores_s, "bo-",color='#857376')
plt.xticks([2, 3, 4, 5,6,7,8,9])
plt.title('Silhouette scores vs Number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Silhoutte score')
#plt.savefig('Silhoutte_score.png')
plt.show()

In [ ]:
#### silhouette_score when Cluster= 2
np.random.seed(40)
from sklearn.metrics import silhouette_score

clustering_spectral = KMeans(n_clusters=2)
clustering_spectral.fit(exp_dataset_1)

print('Silhoutte score of K-Mean clustering is ' + str(silhouette_score(exp_dataset_1, clustering_spectral.labels_)))

In [ ]:
np.random.seed(9)
spectral_labels=clustering_spectral.labels_
spectral_labels

In [ ]:
exp_dataset_1=exp_dataset_1.reset_index(drop='True')

In [ ]:
spectral_final_exp = pd.concat([exp_dataset_1, pd.DataFrame({'pca_clusters':spectral_labels})], axis=1)
spectral_final_exp

In [ ]:
spectral_final_exp['pca_clusters'].value_counts()

In [ ]:
##### Clustering with all Feature (All engineered Features)
#### When Cluster Number are 2

import seaborn as sns
plt.figure(figsize = (17,7))
sns.scatterplot(spectral_final_exp.iloc[:,0],spectral_final_exp.iloc[:,1], hue=spectral_final_exp['pca_clusters'], palette='viridis', s=70, alpha=0.8).set_title('Distribution of cluster based on Spectral Domain Features (PCA)',fontsize=13)
plt.legend()
#plt.savefig('Cluster_pCA_1_and_3')
plt.show()

#### Type 1

In [ ]:
fal_type_1=spectral_final_exp[spectral_final_exp['pca_clusters']==0]
fal_type_2=spectral_final_exp[spectral_final_exp['pca_clusters']==1]

In [ ]:
fal_type_1

In [ ]:
fal_type_1_index=fal_type_1.index

In [ ]:
fal_type_1_index

In [ ]:
only_pred_fal_gmm_reset=only_pred_fal_gmm.reset_index(drop='True')

In [ ]:
len(only_pred_fal_gmm_reset)

In [ ]:
type_1_ids=[]
for i in fal_type_1_index:
    #print(i)
    type_1_ids.append(only_pred_fal_gmm_reset.loc[i])
type_1_ids=pd.DataFrame(type_1_ids)

In [ ]:
type_1_ids

In [ ]:
failure_type1_paths=[]
for index, row in type_1_ids.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    failure_type1_paths.append([CFG_Path, dAT_path])

In [ ]:
len(failure_type1_paths)

In [ ]:
type_1_phases=[]
for index, row in type_1_ids.iterrows():
    type_1_phases.append(row['Phase'])

In [ ]:
len(type_1_phases)

In [ ]:
type_1_dataset=data_frames_creation(failure_type1_paths)

In [ ]:
len(type_1_dataset)

In [ ]:
pred_fal_good_Visualization(type_1_dataset,type_1_phases,col_names)

#### Type 2

In [ ]:
fal_type_2

In [ ]:
fal_type_2_index=fal_type_2.index

In [ ]:
type_2_ids=[]
for i in fal_type_2_index:
    #print(i)
    type_2_ids.append(only_pred_fal_gmm_reset.loc[i])
type_2_ids=pd.DataFrame(type_2_ids)

In [ ]:
failure_type2_paths=[]
for index, row in type_2_ids.iterrows():
    dAT_path = getPath(row['Fault ID'], 1)
    CFG_Path = getPath(row['Fault ID'], 2)
    failure_type2_paths.append([CFG_Path, dAT_path])

In [ ]:
len(failure_type2_paths)

In [ ]:
type_2_phases=[]
for index, row in type_2_ids.iterrows():
    type_2_phases.append(row['Phase'])

In [ ]:
len(type_2_phases)

In [ ]:
type_2_dataset=data_frames_creation(failure_type2_paths)

In [ ]:
pred_fal_good_Visualization(type_2_dataset,type_2_phases,col_names)